## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Altamira,BR,-3.2033,-52.2064,83.52,57,98,0.98,overcast clouds
1,1,Pula,HR,44.8683,13.8481,76.57,78,0,4.61,clear sky
2,2,Pacific Grove,US,36.6177,-121.9166,60.37,78,17,11.86,few clouds
3,3,Touros,BR,-5.1989,-35.4608,76.98,82,62,16.42,broken clouds
4,4,Iqaluit,CA,63.7506,-68.5145,47.70,87,100,4.61,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.reset_index(drop=True)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Altamira,BR,-3.2033,-52.2064,83.52,57,98,0.98,overcast clouds
1,1,Pula,HR,44.8683,13.8481,76.57,78,0,4.61,clear sky
2,3,Touros,BR,-5.1989,-35.4608,76.98,82,62,16.42,broken clouds
3,8,Hilo,US,19.7297,-155.0900,87.28,89,100,3.44,overcast clouds
4,11,Amapa,BR,1.0000,-52.0000,78.37,79,93,1.66,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
209,649,Progreso,MX,21.2833,-89.6667,85.08,71,100,19.13,overcast clouds
210,650,Acarau,BR,-2.8856,-40.1200,79.07,76,48,14.92,scattered clouds
211,658,Bandarbeyla,SO,9.4942,50.8122,76.15,83,28,30.24,scattered clouds
212,663,Katsuura,JP,35.1333,140.3000,79.05,83,98,12.06,overcast clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 0 to 664
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              214 non-null    int64  
 1   City                 214 non-null    object 
 2   Country              214 non-null    object 
 3   Lat                  214 non-null    float64
 4   Lng                  214 non-null    float64
 5   Max Temp             214 non-null    float64
 6   Humidity             214 non-null    int64  
 7   Cloudiness           214 non-null    int64  
 8   Wind Speed           214 non-null    float64
 9   Current Description  214 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 18.4+ KB


In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df=hotel_df.reset_index(drop=True)
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altamira,BR,83.52,overcast clouds,-3.2033,-52.2064,
1,Pula,HR,76.57,clear sky,44.8683,13.8481,
2,Touros,BR,76.98,broken clouds,-5.1989,-35.4608,
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,
4,Amapa,BR,78.37,overcast clouds,1.0000,-52.0000,
5,Bonthe,SL,77.09,overcast clouds,7.5264,-12.5050,
6,Georgetown,MY,78.73,scattered clouds,5.4112,100.3354,
7,Fukue,JP,78.98,light rain,32.6881,128.8419,
8,Butaritari,KI,81.79,few clouds,3.0707,172.7902,
9,Atuona,PF,77.54,scattered clouds,-9.8000,-139.0333,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.drop(labels=index, axis=0)
        print("Hotel not found....skipping.")
        
hotel_df.head(10)
        

Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altamira,BR,83.52,overcast clouds,-3.2033,-52.2064,Requinte Hotel
1,Pula,HR,76.57,clear sky,44.8683,13.8481,Park Plaza Histria Pula
2,Touros,BR,76.98,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Amapa,BR,78.37,overcast clouds,1.0000,-52.0000,
5,Bonthe,SL,77.09,overcast clouds,7.5264,-12.5050,Bonthe Holiday Village
6,Georgetown,MY,78.73,scattered clouds,5.4112,100.3354,Cititel Penang
7,Fukue,JP,78.98,light rain,32.6881,128.8419,Goto Guesthouse Utojuku & SA Car Rental
8,Butaritari,KI,81.79,few clouds,3.0707,172.7902,Isles Sunset Lodge
9,Atuona,PF,77.54,scattered clouds,-9.8000,-139.0333,Villa Enata


In [25]:
clean_hotel_df = hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng", "Hotel Name"]].copy()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altamira,BR,83.52,overcast clouds,-3.2033,-52.2064,Requinte Hotel
1,Pula,HR,76.57,clear sky,44.8683,13.8481,Park Plaza Histria Pula
2,Touros,BR,76.98,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Amapa,BR,78.37,overcast clouds,1.0000,-52.0000,
...,...,...,...,...,...,...,...
209,Progreso,MX,85.08,overcast clouds,21.2833,-89.6667,Playa Linda Hotel
210,Acarau,BR,79.07,scattered clouds,-2.8856,-40.1200,castelo pizzaria
211,Bandarbeyla,SO,76.15,scattered clouds,9.4942,50.8122,JABIR HOTEL
212,Katsuura,JP,79.05,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki


In [26]:
clean_hotel_df=clean_hotel_df[clean_hotel_df["Hotel Name"].notna()]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altamira,BR,83.52,overcast clouds,-3.2033,-52.2064,Requinte Hotel
1,Pula,HR,76.57,clear sky,44.8683,13.8481,Park Plaza Histria Pula
2,Touros,BR,76.98,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Amapa,BR,78.37,overcast clouds,1.0000,-52.0000,
...,...,...,...,...,...,...,...
209,Progreso,MX,85.08,overcast clouds,21.2833,-89.6667,Playa Linda Hotel
210,Acarau,BR,79.07,scattered clouds,-2.8856,-40.1200,castelo pizzaria
211,Bandarbeyla,SO,76.15,scattered clouds,9.4942,50.8122,JABIR HOTEL
212,Katsuura,JP,79.05,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki


In [27]:
#7a. replace empty string with NaN
clean_hotel_df.replace(to_replace='',value=np.nan,regex=True)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altamira,BR,83.52,overcast clouds,-3.2033,-52.2064,Requinte Hotel
1,Pula,HR,76.57,clear sky,44.8683,13.8481,Park Plaza Histria Pula
2,Touros,BR,76.98,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Amapa,BR,78.37,overcast clouds,1.0000,-52.0000,NaN
...,...,...,...,...,...,...,...
209,Progreso,MX,85.08,overcast clouds,21.2833,-89.6667,Playa Linda Hotel
210,Acarau,BR,79.07,scattered clouds,-2.8856,-40.1200,castelo pizzaria
211,Bandarbeyla,SO,76.15,scattered clouds,9.4942,50.8122,JABIR HOTEL
212,Katsuura,JP,79.05,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki


In [28]:
clean_hotel_df=clean_hotel_df[clean_hotel_df["Hotel Name"].notna()]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altamira,BR,83.52,overcast clouds,-3.2033,-52.2064,Requinte Hotel
1,Pula,HR,76.57,clear sky,44.8683,13.8481,Park Plaza Histria Pula
2,Touros,BR,76.98,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Amapa,BR,78.37,overcast clouds,1.0000,-52.0000,
...,...,...,...,...,...,...,...
209,Progreso,MX,85.08,overcast clouds,21.2833,-89.6667,Playa Linda Hotel
210,Acarau,BR,79.07,scattered clouds,-2.8856,-40.1200,castelo pizzaria
211,Bandarbeyla,SO,76.15,scattered clouds,9.4942,50.8122,JABIR HOTEL
212,Katsuura,JP,79.05,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki


In [29]:
# 7. Drop the rows where there is no Hotel Name.
cleaned_hotel_df=clean_hotel_df.dropna()
cleaned_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altamira,BR,83.52,overcast clouds,-3.2033,-52.2064,Requinte Hotel
1,Pula,HR,76.57,clear sky,44.8683,13.8481,Park Plaza Histria Pula
2,Touros,BR,76.98,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Amapa,BR,78.37,overcast clouds,1.0000,-52.0000,
...,...,...,...,...,...,...,...
209,Progreso,MX,85.08,overcast clouds,21.2833,-89.6667,Playa Linda Hotel
210,Acarau,BR,79.07,scattered clouds,-2.8856,-40.1200,castelo pizzaria
211,Bandarbeyla,SO,76.15,scattered clouds,9.4942,50.8122,JABIR HOTEL
212,Katsuura,JP,79.05,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki


# I spent about an hour on askBSC with Sunshine Pavelka trying to figure out why the code wasn't working to drop the NaN's.  You can see the different codes we tried, but no matter what we did, it just turned the NaN's back to blank strings.  

In [30]:
# 8a. Create the output File (CSV)
output_data_file="../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
cleaned_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Lng"]]

In [32]:
locations

,Lat,Lng
0,-3.2033,-52.2064
1,44.8683,13.8481
2,-5.1989,-35.4608
3,19.7297,-155.0900
4,1.0000,-52.0000
...,...,...
209,21.2833,-89.6667
210,-2.8856,-40.1200
211,9.4942,50.8122
212,35.1333,140.3000


In [33]:
# 11a. Add a marker layer for each city to the map. 
locations=hotel_df[["Lat", "Lng"]]
hotel=hotel_df["Hotel Name"]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))